In [1]:
import telebot
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
import yfinance as yf
import pandas as pd
from catboost import CatBoost, CatBoostRegressor, Pool
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error as mape
import numpy as np
import holidays
import warnings
warnings.filterwarnings("ignore")
MIN_RETRO = 1
MAX_RETRO = 10
SPLITS = 800
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Input, concatenate, Embedding, Reshape, BatchNormalization,Dropout
import tensorflow.keras.backend as K
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import datetime
from datetime import timedelta
from sqlalchemy import create_engine,text
import re
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

from datetime import timedelta
import calendar
import datetime
from mlbase.db import MLData,Table
from bot_brain import pred
db = MLData('postgres',user='seb',host='127.0.0.1')
bot = telebot.TeleBot("key", parse_mode=None)


2023-08-14 15:29:58.559177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 15:29:58.617845: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-14 15:29:58.620489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-14 15:29:58.620498: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [2]:
def translate_weekday_eng_to_rus(weekday):
    weekdays_eng = list(calendar.day_name)
    weekdays_rus = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
    return weekdays_rus[weekdays_eng.index(weekday)]

In [3]:
def zero(cid):
    bot.send_message(cid, f"Здраствуйте! Меня зовут Бот-Клиник могу записать к врачу 24/7 и перенести запись!")
    bot.send_message(cid, f"Мне нужен Ваш Ф.И.О, что продолжить. Как Вас зовут?")
    df  = pd.DataFrame({'cid':[968235182],'state':[-1]})
    Table('clinic_bot.states',db=db).copy_df(df,'append')
    
    db.cur.execute(f'update clinic_bot.states set state  = -1 where cid = {cid}')
    db.commit()

In [4]:
def get_name(cid,message):

    try:
        segmenter = Segmenter()
        morph_vocab = MorphVocab()
        text = message
        emb = NewsEmbedding()
        morph_tagger = NewsMorphTagger(emb)
        syntax_parser = NewsSyntaxParser(emb)
        ner_tagger = NewsNERTagger(emb)

        names_extractor = NamesExtractor(morph_vocab)
        dates_extractor = DatesExtractor(morph_vocab)
        money_extractor = MoneyExtractor(morph_vocab)
        addr_extractor = AddrExtractor(morph_vocab)
        doc = Doc(text)
        doc.segment(segmenter) 
        doc.tag_morph(morph_tagger)
        doc.parse_syntax(syntax_parser)
        doc.tag_ner(ner_tagger)

        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)

        for span in doc.spans:
            span.normalize(morph_vocab)

        {_.text: _.normal for _ in doc.spans if _.text != _.normal}
        for token in doc.tokens:
            token.lemmatize(morph_vocab)

        {_.text: _.lemma for _ in doc.tokens}
        matches = dates_extractor(text)
        facts = [i.fact.as_json for i in matches]
        for f in facts:
            print(f"{f.get('day')}.{f.get('month')}.{f.get('year')}")
        for span in doc.spans:
            if span.type == PER:
                span.extract_fact(names_extractor)

        names_dict = {_.normal: _.fact.as_dict for _ in doc.spans if _.fact}
        print(names_dict)
        if len(names_dict.keys())>0:
            print('pepe')
            for i in names_dict.keys():

                full_name = i
                break

            name = names_dict[full_name]['first']
            

            bot.send_message(cid, f"Вас зовут {name}?")
            print(f"Вас зовут {name}?")
            db.cur.execute(f'update clinic_bot.states set state  = -2 where cid = {cid}')
            db.commit()
            db.cur.execute(f"""update clinic_bot.users set name  = '{name}' where cid = {cid}""")
            db.commit()
            return 0
            print(f"Не понял Вас, пожалуйста напишите снова")
    except Exception as e:
        print(message)
        print(doc.spans)
        print(doc)
        print(e)
    bot.send_message(cid, f"Не понял Вас, пожалуйста напишите снова")


    

In [5]:
def get_last_name(cid,message):


    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    text = message
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)
    ner_tagger = NewsNERTagger(emb)

    names_extractor = NamesExtractor(morph_vocab)
    dates_extractor = DatesExtractor(morph_vocab)
    money_extractor = MoneyExtractor(morph_vocab)
    addr_extractor = AddrExtractor(morph_vocab)
    doc = Doc(text)
    doc.segment(segmenter) 
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    for span in doc.spans:
        span.normalize(morph_vocab)

    {_.text: _.normal for _ in doc.spans if _.text != _.normal}
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    {_.text: _.lemma for _ in doc.tokens}
    matches = dates_extractor(text)
    facts = [i.fact.as_json for i in matches]
    for f in facts:
        print(f"{f.get('day')}.{f.get('month')}.{f.get('year')}")
    for span in doc.spans:
        if span.type == PER:
            span.extract_fact(names_extractor)
    names_dict = {_.normal: _.fact.as_dict for _ in doc.spans if _.fact}
    if len(names_dict.keys())>0:
            print('pepe')
            for i in names_dict.keys():
                full_name = i
                break
    return names_dict[full_name]['last']
 
        

In [6]:
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    print(message.json)
    cid = message.json['from']['id']
    try:
        df = pd.read_sql(f"""select count(*) c  from clinic_bot.states where cid = {cid}""",eng).c.tolist()[0]
        df_1 = pd.read_sql(f"""select count(*) c  from clinic_bot.users where cid = {cid}""",eng).c.tolist()[0]
        
        if df > 0 and df_1 > 0:
            name = pd.read_sql(f"""select name  from clinic_bot.users where cid = {cid}""",db.engine).name.tolist()[0]
        
            if str(name) == 'None':
                zero(cid)
                db.cur.execute(f'update clinic_bot.states set state  = -1 where cid = {cid}')
                db.commit()

                return 0
            else:
                bot.send_message(cid, f"Добрый день, {name}!\n. В чём могу Вам помочь? К какому вречу интересует записаться?")
                return 0

        else:
            tmp = pd.DataFrame({'cid':[cid],'name':['']})
            Table('clinic_bot.users',db=db).copy_df(tmp,'append')
            tmp = pd.DataFrame({'cid':[cid],'state':[-1]})
            Table('clinic_bot.states',db=db).copy_df(tmp,'append')
        
            zero(cid)
            
    except:
        tmp = pd.DataFrame({'cid':[cid],'name':['']})
        Table('clinic_bot.users',db=db).copy_df(tmp,'append')
        zero(cid)

           
    return 0
    

In [7]:
@bot.message_handler(commands=['delete'])
def delete_user(message):
    cid = message.json['from']['id']
    try:
        db.cur.execute(f'''DELETE FROM  clinic_bot.users  WHERE  cid = {cid}''')
        db.commit()
        db.cur.execute(f'''DELETE FROM  clinic_bot.states  WHERE  cid = {cid}''')
        db.commit()
        db.cur.execute(f'''DELETE FROM   clinic_bot.wait_confirm  WHERE  cid = {cid}''')
        db.commit()
        db.cur.execute(f'''DELETE FROM   clinic_bot.dates  WHERE  cid = {cid}''')
        db.commit()
      
    except:
        db.commit()

In [8]:
def  model_aff(cid,message):

    if pred(message) == 'yes':
        name  = pd.read_sql(f"""select name from   clinic_bot.users where cid = {cid}""",db.engine).name.tolist()[0]
        sts = pd.read_sql(f'select state from  clinic_bot.states where cid = {cid}',db.engine)['state'].tolist()[0]
        if sts == -2:
            bot.send_message(cid, f"Очень приятно {name}!\n. В чём могу Вам помочь? К какому врaчу интересует записаться?")
        return 0

    elif pred(message) == 'no':
        bot.send_message(cid, f"Пожалуйста напишите снова, тогда")
    else:
        bot.send_message(cid, f"Не понял, напишите снова, пожалуйста")
        
    return -1

In [9]:
def model_info(cid,message):
    answ = pred(message)

    if 'question_' in answ or 'cancel' in answ or 'change' in answ:
        if 'question_' in answ:
            if 'question_doc' == answ:
                bot.send_message(cid, f"К какому доктору именно интересует?")
            else:
                doc = pd.read_sql(f"""select * from  clinic_bot.sp where tar = '{answ}' """,db.engine)
                bot.send_message(cid, f"{doc['Специалитет'].tolist()[0]}?")
                db.cur.execute(f'update clinic_bot.states set state  = -3 where cid = {cid}')
                db.commit()
                df = pd.DataFrame({"cid":[cid],'sp_id':[doc['sp_id'].tolist()[0]]})
                Table('clinic_bot.wait_confirm',db=db).copy_df(df,'append')
        else:
            if answ == 'cancel':
                
            
                bot.send_message(cid,'Понял, хотите отменить запись')
                if pre_cancel(cid) ==1:
                    db.cur.execute(f'update clinic_bot.states set state  = 4 where cid = {cid}')
                    db.commit()
                
            elif answ == 'change':
                bot.send_message(cid,'Понимаю что хотите перенести запись но пока не сделано этот функционал')
                
            
    else:
        bot.send_message(cid, f"Я Вас не понял, напишите снова пожалуйста, в чем могу помочь")
        

In [10]:
import pandas as pd

def generate_calendar():
    hours = pd.date_range('6:00', '21:00', freq='30min').strftime('%H:%M').astype(str)
    
    weekdays = pd.date_range(start='2023-08-13', end='2023-12-31', freq='D')
    num_ones = 8
    calendar = pd.DataFrame(index=weekdays, columns=hours).fillna(0)
    for i,j in enumerate(calendar.index):
        for _ in range(3):
            
            row = calendar.iloc[i]
            random_column = random.choice(row.index)
            calendar.at[j, random_column] = 1

    calendar['weekday'] = calendar.index.weekday.map(get_day_of_week)
    calendar = calendar.reset_index().rename(columns={'index':'date'})
    return calendar

In [11]:
def get_day_of_week(date):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return days[date]

In [12]:
def cancel(cid,doc_id):
    db.cur.execute(f"DELETE FROM  clinic_bot.dates where cid = {cid} and doc_id = {doc_id} ")
    db.commit()
    bot.send_message(cid, "Отменил!:)")

In [13]:
pd.read_sql('select * from   clinic_bot.docs',db.engine).columns

Index(['doc_id', 'Фамилия', 'Имя', 'Отчество', 'sp_id'], dtype='object')

In [14]:
def pre_cancel(cid):
   
    df = pd.read_sql(f"select count(*) c from   clinic_bot.dates where cid = {cid}",db.engine).c.tolist()[0]
   
    if df > 0:
        if df >1:
            msm = 'Какой запись хотите отменить, напишите Фамилия врача\n'
        else:
            msm = 'Этот запись? \n'
        df = pd.read_sql(f"select * from   clinic_bot.dates o  left join clinic_bot.docs r on r.doc_id = o.doc_id where cid = {cid}",db.engine)
        for last_name,name,otch,date,hour in  zip(df['Фамилия'],df['Имя'],df['Отчество'],df['date'],df['hour']):
            msm += f'Врач:{last_name} {name} {otch} Дата и время: {date} {hour}\n'
        bot.send_message(cid, msm)
        if len(df) ==1:
            db.cur.execute(f'update clinic_bot.states set state  = -6 where cid = {cid}')
            db.commit()
        else:
            db.cur.execute(f'update clinic_bot.states set state  = -7 where cid = {cid}')
            db.commit()
                
            
            return 0
    else:
        bot.send_message(cid, "У Вас нет записи")
        db.cur.execute(f'update clinic_bot.states set state  = 2 where cid = {cid}')
        db.commit()
            
            
    

In [15]:
def give_options(cid):
    
    sp_id = pd.read_sql(f"""select sp_id from clinic_bot.wait_confirm where cid = {cid}""",db.engine).sp_id.tolist()[0]
    doc = pd.read_sql(f'select * from clinic_bot.docs where sp_id = {sp_id} limit 1',db.engine)
    name  = doc['Имя'].tolist()[0]
    otch = doc['Отчество'].tolist()[0]
    today = datetime.date.today()
    cols = pd.read_sql(f"""select * from clinic_bot.calendar_1 limit 1""",db.engine).columns 
   
    cond = [f""" "{x}"  """   for x in cols[1:-1]]
    one = f"""select *, {'+'.join(cond )} ones from clinic_bot.calendar_{ doc['doc_id'].tolist()[0]}"""
    sql = f"""SELECT * FROM ({one}) sub
    WHERE ones >0
    """
    msm = f'С врачом {name} {otch}\n'
    df = pd.read_sql(sql,db.engine).head(4)
    for row in df.iterrows():
        
        msm += translate_weekday_eng_to_rus(row[1]['weekday']) + '  '+ str(row[1]['date']).split(' ')[0]+'\n'
        for x in cols[1:-1]:
            if row[1][x] > 0 :
                msm += x+'\n'
                
    bot.send_message(cid, msm)
    bot.send_message(cid, "Когда Вам удобно?\n Напишите дату и время с ':', например в 16 октобря 12:30 ")
    db.cur.execute(f'update clinic_bot.states set state  = -4 where cid = {cid}')
    db.commit()
    
    

In [16]:
def get_hour(txt):
    return  re.search(r'\d{1,2}:\d{2}', fr"{txt}")[0]

In [17]:
def get_date(text):

    segmenter = Segmenter()
    morph_vocab = MorphVocab()


    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)
    ner_tagger = NewsNERTagger(emb)

    names_extractor = NamesExtractor(morph_vocab)
    dates_extractor = DatesExtractor(morph_vocab)
    money_extractor = MoneyExtractor(morph_vocab)
    addr_extractor = AddrExtractor(morph_vocab)
    doc = Doc(text)
    doc.segment(segmenter) 
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    for span in doc.spans:
        span.normalize(morph_vocab)

    {_.text: _.normal for _ in doc.spans if _.text != _.normal}
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    {_.text: _.lemma for _ in doc.tokens}
    matches = dates_extractor(text)
    facts = [i.fact.as_json for i in matches]
    day,year,month=0,0,0
    for f in facts:
        day = f.get('day')
        month  =f.get('month')
        year = f.get('year')
    return day,month,year

In [18]:
def set_date(cid,date,hour):

    cols = pd.read_sql(f"""select * from clinic_bot.calendar_1 limit 1""",db.engine).columns 
    if len(str(hour).split(':')[0]) ==1:
        hour = '0'+hour
    if hour not in cols:

        bot.send_message(cid, "Не понял. Напишите, пожалуйста, снова дата и время, например в 16 октобря 12:30 ")
        return 0
    else:
        year = date[2]
        if str(date[2]) == 'None':
            year = 2023

        if date[1] == 0 or date[0] == 0:
            bot.send_message(cid, "Не понял. Напишите, пожалуйста, снова дата и время, например в 16 октобря 12:30 ")
            return 0 
        date_string = f'{year}-{date[1]}-{date[0]}'

        date = datetime.datetime.strptime(date_string, '%Y-%m-%d')
        date = str(date).split(' ')[0]
        sp_id = pd.read_sql(f"""select sp_id from clinic_bot.wait_confirm where cid = {cid}""",db.engine).sp_id.tolist()[0]

        doc_id = pd.read_sql(f'select * from clinic_bot.docs where sp_id = {sp_id} limit 1',db.engine)['doc_id'].tolist()[0]
        df = pd.read_sql(f"""select count(*) co from clinic_bot.calendar_{doc_id} where date = '{date}' and "{hour}" >0  
                            """,db.engine).co.tolist()[0]
        if df > 0:
            sp_id = pd.read_sql(f"""select sp_id from clinic_bot.wait_confirm where cid = {cid}""",db.engine).sp_id.tolist()[0]

            doc = pd.read_sql(f'select * from clinic_bot.docs where sp_id = {sp_id} limit 1',db.engine)['doc_id'].tolist()[0]
            df = pd.DataFrame({'date':[date],'cid':[cid],'doc_id':[doc],'flag':[1],'hour':[hour]})
            Table('clinic_bot.dates',db=db).copy_df(df,'append')
            bot.send_message(cid, f"Получается запишем Вас {date} в {hour} часов.\n Потдверждайте?")
            db.cur.execute(f'update clinic_bot.states set state  = -5 where cid = {cid}')
            db.commit()
            return 0
        else:
            bot.send_message(cid, "Такое время не доступнио. Напишите, пожалуйста, снова дата и время, например в 16 октобря 12:30 ")
            return 0 



In [21]:
@bot.message_handler(func=lambda m: True)
def echo_all(message):

    cid = message.json['from']['id']
    df = pd.read_sql(f"""select count(*) c  from clinic_bot.users where cid = {cid}""",eng).c.tolist()[0]

    if df > 0:

        state  = pd.read_sql(f'select state from clinic_bot.states where cid = {cid}',db.engine).state.tolist()[0]
        if state == -1:
            get_name(cid,message.text)
            return 0
        name = pd.read_sql(f"""select name  from clinic_bot.users where cid = {cid}""",db.engine).name.tolist()[0]
        print(state)
        
        if state == -2:

            if model_aff(cid,message.text) == 0 :
                db.cur.execute(f'update clinic_bot.states set state  = 2 where cid = {cid}')
                db.commit()
            return 0
        elif state == -3:
            if model_aff(cid,message.text) == 0 :
                db.cur.execute(f'update clinic_bot.states set state  = 3 where cid = {cid}')
                db.commit()
                give_options(cid)
                return
            db.cur.execute(f'update clinic_bot.states set state  = 2 where cid = {cid}')
            db.commit()
            return 0
        elif state == -4:
            try:
                txt = message.text
                date = get_date(txt)
                hour =  get_hour(txt)
                print(date,hour)
                set_date(cid,date,hour)
            except:
                bot.send_message(cid, "Не понял. Напишите, пожалуйста, снова дата и время, например в 16 октобря 12:30 ")
                
            return 0
        elif state == -5:
            if model_aff(cid,message.text) == 0 :
                df = pd.read_sql(f'select * from clinic_bot.dates where cid = {cid} and flag = 1 limit 1',db.engine)
                doc_id = df['doc_id'].tolist()[0]
                hour = df['hour'].tolist()[0]
                date = df['date'].tolist()[0]
                
                db.cur.execute(f"""update clinic_bot.calendar_{doc_id} set "{hour}"= 0 where date = '{date}' """)
                db.commit()
                
                db.cur.execute(f'update clinic_bot.states set state  = 1 where cid = {cid}')
                db.commit()
                bot.send_message(cid, f"Отлично! Ждём Вас")

                db.cur.execute(f'update clinic_bot.dates set flag  = 0 where cid = {cid}')
                db.commit()
                clinic_bot.wait_confirm
                db.cur.execute(f'delete from clinic_bot.wait_confirm where cid = {cid}')
                db.commit()
        
            else:
                db.cur.execute(f'update clinic_bot.states set state  = -4 where cid = {cid}')
                db.commit()
                db.cur.execute(f'delete from clinic_bot.dates where cid = {cid} and flag = 1')
                db.commit()
            return 0
        elif state == -6:
            if model_aff(cid,message.text) == 0 :
                doc_id = pd.read_sql(f'select doc_id from  clinic_bot.dates where cid = {cid}',db.engine)['doc_id'].tolist()[0]
                print(doc_id)
                cancel(cid,doc_id)
                db.cur.execute(f'update clinic_bot.states set state  = 1 where cid = {cid}')
                db.commit()
            else:
                db.cur.execute(f'update clinic_bot.states set state  = 1 where cid = {cid}')
                db.commit()
                bot.send_message(cid, f"У вас только такой запись")
            return 0
        elif state == -7:
            try:
                last = get_last_name(cid,message.text)
                print(last)
                doc_id = pd.read_sql(f"""select * from clinic_bot.docs where "Фамилия"  = '{last}'  """,db.engine).doc_id.tolist()[0]
                db.cur.execute(f'update clinic_bot.states set state  = 1 where cid = {cid}')
                db.commit()
                print(f'select hour,date from clinic_bot.dates where doc_id = {doc_id} and cid = {cid}')
                df = pd.read_sql(f'select hour,date from clinic_bot.dates where doc_id = {doc_id} and cid = {cid}',db.engine)
                hour = df['hour'].tolist()[0]
                date = df['date'].tolist()[0]
                db.cur.execute(f"""update clinic_bot.calendar_{doc_id} set "{hour}"  = 1 where date = '{date}'   """)
                db.commit()

                cancel(cid,doc_id)

            except:
                bot.send_message(cid, f"Нет врач с такой фамилой, напишите снова пожалуйста")
            return 0
        elif state == 1:
            name = pd.read_sql(f"""select name  from clinic_bot.users where cid = {cid}""",db.engine).name.tolist()[0]

            bot.send_message(cid, f"Добрый день, {name}!\n. В чём могу Вам помочь? К какому вречу интересует записаться?")
            
            db.cur.execute(f'update clinic_bot.states set state  = 2 where cid = {cid}')
            db.commit()
            return 0
        elif state == 2:
            model_info(cid,message.text)
        elif state == 3:
            give_options(cid)
            return 0
        elif state == 4:
            pre_cancel(cid)
            
        


In [22]:
bot.infinity_polling()

2023-08-14 17:07:09,472 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-08-14 17:07:09,473 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


In [ ]:
pd.read_sql(f"""select * from clinic_bot.docs where "Фамилия"  = 'Себякин'  """,db.engine)

In [ ]:
pd.read_sql(f'pd.read_sql(f'select hour,date from clinic_bot.dates where doc_id = {doc_id} and cid = {cid}',db.engine)',db.engine)

In [ ]:
db.cur.execute(f'update clinic_bot.calendar_{doc_id} set "{hour}"  = 1 where date = '{date}'   """)
db.commit()